In [5]:
from datasets import load_dataset

ds = load_dataset("hpe-ai/medical-cases-classification-tutorial")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
for split in ds:
    ds[split].save_to_disk(f"./medical_cases_{split}")

Saving the dataset (0/1 shards):   0%|          | 0/1724 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/370 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/370 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType
import torch

# -------------------------------------------------------------------
# 1. LOAD & PREPROCESS THE DATA
# -------------------------------------------------------------------

# Load the CSV dataset
df = pd.read_csv(r"medical_cases_train\medical_cases_train.csv")

# Keep only the relevant columns and drop rows with missing values
df = df[["description", "transcription"]].dropna()

# Convert the Pandas dataframe into a Hugging Face Dataset object
dataset = Dataset.from_pandas(df)

# Format each row as a prompt-response dialogue (chat style for Gemma)
def format_prompt(example):
    return {
        "text": f"<start_of_turn>user\n{example['description']}\n<end_of_turn>\n<start_of_turn>model\n{example['transcription']}<end_of_turn>"
    }

# Apply the prompt formatting to each row in the dataset
dataset = dataset.map(format_prompt)

# -------------------------------------------------------------------
# 2. LOAD TOKENIZER AND BASE MODEL
# -------------------------------------------------------------------

# Set the name of the model checkpoint
model_name = "google/gemma-3-1b-it"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure that padding token is set (Gemma uses EOS token for padding too)
tokenizer.pad_token = tokenizer.eos_token

# Load the model in float16 precision and place it on available GPU automatically
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use 16-bit precision (efficient on modern GPUs)
    device_map="auto"  # Automatically spreads model across available GPUs
)

# -------------------------------------------------------------------
# 3. APPLY LoRA FOR PARAMETER-EFFICIENT FINE-TUNING
# -------------------------------------------------------------------

# Define the LoRA configuration (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=8,  # Rank of the update matrices (lower = less compute, higher = more expressive)
    lora_alpha=32,  # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"],  # These are the attention projection layers to inject LoRA into
    lora_dropout=0.05,  # Dropout for regularization
    bias="none",  # Do not adapt the biases
    task_type=TaskType.CAUSAL_LM  # Task is causal language modeling (predict next token)
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

# -------------------------------------------------------------------
# 4. TOKENIZE THE DATA
# -------------------------------------------------------------------

# Tokenization function that also prepares labels (for training)
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512  # You can adjust this based on GPU memory
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Set labels for language modeling
    return tokens

# Apply tokenization
tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# -------------------------------------------------------------------
# 5. SETUP TRAINING CONFIGURATION
# -------------------------------------------------------------------

training_args = TrainingArguments(
    output_dir="./gemma-lora-medical",  # Directory to save model checkpoints
    per_device_train_batch_size=1,  # Use batch size 1 (can be increased if GPU allows)
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    num_train_epochs=3,  # Train for 3 epochs
    learning_rate=2e-4,  # Learning rate (tuned for LoRA)
    fp16=True,  # Enable mixed precision training
    logging_dir="./logs",  # Where to write logs
    save_strategy="epoch",  # Save model every epoch
    save_total_limit=2  # Keep only the latest 2 checkpoints
)

# -------------------------------------------------------------------
# 6. INITIALIZE THE TRAINER
# -------------------------------------------------------------------

trainer = Trainer(
    model=model,  # LoRA-wrapped model
    args=training_args,
    train_dataset=tokenized_dataset,  # Tokenized dataset
    tokenizer=tokenizer,  # Needed for padding and decoding
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # MLM=False for causal LM
)

# -------------------------------------------------------------------
# 7. START TRAINING
# -------------------------------------------------------------------

trainer.train()

# -------------------------------------------------------------------
# 8. SAVE THE FINE-TUNED MODEL AND TOKENIZER
# -------------------------------------------------------------------

model.save_pretrained("./gemma-lora-medical")
tokenizer.save_pretrained("./gemma-lora-medical")


Map:   0%|          | 0/1724 [00:00<?, ? examples/s]

model.safetensors:  78%|#######7  | 1.55G/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Map:   0%|          | 0/1724 [00:00<?, ? examples/s]

C:\Users\abhay\AppData\Local\Temp\ipykernel_20236\2597363362.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhayvg-2904 (abhayvg-2904-indian-institute-of-technology-gandhinagar). Use `wandb login --relogin` to force relogin


It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss


In [ ]:
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer, EvalPrediction
import torch
from peft import LoraConfig
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -------------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------------
train_df = pd.read_csv("medical_cases_train/medical_cases_train.csv")[["description", "transcription"]].dropna()
val_df = pd.read_csv("medical_cases_validation/medical_cases_validation.csv")[["description", "transcription"]].dropna()
test_df = pd.read_csv("medical_cases_test/medical_cases_test.csv")[["description", "transcription"]].dropna()

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# -------------------------------------------------------------------
# 2. FORMAT PROMPTS
# -------------------------------------------------------------------
def format_prompt(example):
    return {
        "text": f"<start_of_turn>user\n{example['description']}\n<end_of_turn>\n<start_of_turn>model\n{example['transcription']}<end_of_turn>"
    }

train_dataset = train_dataset.map(format_prompt)
val_dataset = val_dataset.map(format_prompt)
test_dataset = test_dataset.map(format_prompt)

# -------------------------------------------------------------------
# 3. LOAD MODEL
# -------------------------------------------------------------------
model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True
)
tokenizer.pad_token = tokenizer.eos_token

# -------------------------------------------------------------------
# 4. APPLY LoRA
# -------------------------------------------------------------------
FastLanguageModel.for_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model.add_adapter(lora_config)

# -------------------------------------------------------------------
# 5. TOKENIZATION
# -------------------------------------------------------------------
def tokenize(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

train_dataset = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize, remove_columns=test_dataset.column_names)

# -------------------------------------------------------------------
# 6. TRAINING ARGUMENTS
# -------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./gemma-lora-medical",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=6,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)


# -------------------------------------------------------------------
# 7. METRICS FUNCTION
# -------------------------------------------------------------------
def compute_metrics(eval_pred: EvalPrediction):
    preds = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids

    # Flatten and ignore padded tokens
    true_labels = []
    pred_labels = []
    for pred, label in zip(preds, labels):
        for p, l in zip(pred, label):
            if l != -100:
                true_labels.append(l)
                pred_labels.append(p)

    return {
        "accuracy": accuracy_score(true_labels, pred_labels),
        "precision": precision_score(true_labels, pred_labels, average='macro', zero_division=0),
        "recall": recall_score(true_labels, pred_labels, average='macro', zero_division=0),
        "f1": f1_score(true_labels, pred_labels, average='macro', zero_division=0),
    }

# -------------------------------------------------------------------
# 8. TRAINER
# -------------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# -------------------------------------------------------------------
# 9. FINAL TEST EVALUATION
# -------------------------------------------------------------------
print("\n=== Final Evaluation on Test Set ===")
test_results = trainer.evaluate(eval_dataset=test_dataset)
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")

# -------------------------------------------------------------------
# 10. SAVE MODEL
# -------------------------------------------------------------------
model.save_pretrained("./gemma-lora-medical")
tokenizer.save_pretrained("./gemma-lora-medical")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


Map:   0%|          | 0/1724 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

c:\Users\abhay\anaconda3\envs\assignment1\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


...

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

C:\Users\abhay\AppData\Local\Temp\ipykernel_9432\3614802169.py:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,724 | Num Epochs = 6 | Total steps = 1,290
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 745,472/1,000,000,000 (0.07% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.959900
20,3.015200
30,2.610200
40,2.504900
50,2.491600
60,2.387100
70,2.422100
80,2.339800
90,2.210100
100,2.277900


In [ ]:
import pandas as pd
from transformers import AutoTokenizer
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load model and tokenizer
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="gemma-lora-medical/checkpoint-1290",
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True
)
model.eval()

# Load test data
test_df = pd.read_csv("medical_cases_test/medical_cases_test.csv")
test_df = test_df.dropna(subset=["sample_name", "medical_specialty"])

# Dynamically extract unique medical specialties (target classes)
target_classes = sorted(test_df["medical_specialty"].unique())
target_classes_lower = [cls.lower() for cls in target_classes]

# Predictions
pred_labels = []
true_labels = []

print("\n=== Predictions on Test Set ===\n")

for i in tqdm(range(len(test_df))):
    sample_name = test_df.iloc[i]["sample_name"]
    true_label = test_df.iloc[i]["medical_specialty"]

    prompt = f"""
Classify the following medical sample into one of the following medical specialties:
{target_classes}

Sample Name: {sample_name}
Medical Specialty:
"""

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=20,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction_raw = decoded.split("Medical Specialty:")[-1].strip()

    # Match prediction to one of the known target classes
    matched_class = None
    for cls in target_classes:
        if cls.lower() in prediction_raw.lower():
            matched_class = cls
            break

    if not matched_class:
        for cls in target_classes:
            if any(word in prediction_raw.lower() for word in cls.lower().split()):
                matched_class = cls
                break

    if not matched_class:
        matched_class = "Unknown"

    pred_labels.append(matched_class)
    true_labels.append(true_label)

    print(f"Sample: {sample_name}")
    print(f"Prediction: {matched_class}")
    print(f"True Label: {true_label}\n")

# Filter out 'Unknown' predictions for evaluation
filtered_preds = [p for p in pred_labels if p != "Unknown"]
filtered_truth = [t for p, t in zip(pred_labels, true_labels) if p != "Unknown"]

print("=== Evaluation Metrics (Excluding 'Unknown') ===")
print("Accuracy:", accuracy_score(filtered_truth, filtered_preds))
print("Precision:", precision_score(filtered_truth, filtered_preds, average='macro', zero_division=0))
print("Recall:", recall_score(filtered_truth, filtered_preds, average='macro', zero_division=0))
print("F1 Score:", f1_score(filtered_truth, filtered_preds, average='macro', zero_division=0))


C:\Users\abhay\AppData\Local\Temp\ipykernel_12496\1019256308.py:8: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\abhay\anaconda3\envs\assignment1\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.1.
   \\   /|    NVIDIA GeForce RTX 3050 Ti Laptop GPU. Num GPUs = 1. Max memory: 4.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

=== Predictions on Test Set ===



  0%|          | 1/370 [00:08<52:28,  8.53s/it]

...

Sample: MGUS Followup
Prediction: Cardiovascular / Pulmonary
True Label: Hematology - Oncology



 59%|█████▉    | 219/370 [08:38<08:37,  3.43s/it]

Sample: Carpal Tunnel Release - Endoscopic
Prediction: Unknown
True Label: Orthopedic



 59%|█████▉    | 220/370 [08:39<07:23,  2.96s/it]

Sample: Transesophageal Echocardiogram - 6
Prediction: Cardiovascular / Pulmonary
True Label: Cardiovascular / Pulmonary

